In [7]:
import random

trainval_txt_file_path = 'datasets/kaist-rgbt/trainval-all-04.txt'
train_txt_file_path = 'datasets/kaist-rgbt/train-all-04.txt'
val_txt_file_path = 'datasets/kaist-rgbt/val-all-04.txt'

with open(trainval_txt_file_path, 'r') as file:
    lines = file.readlines()

val_size = len(lines) // 5
indices = sorted(random.sample(range(len(lines)), val_size))

val_lines = [lines[i] for i in indices]
remaining_lines = [line for i, line in enumerate(lines) if i not in indices]

with open(val_txt_file_path, 'w') as file:
    file.writelines(val_lines)

with open(train_txt_file_path, 'w') as file:
    file.writelines(remaining_lines)

print(f'saved {val_size} images in val-all-04.txt')
print(f'saved {len(remaining_lines)} images in train-all-04.txt')


saved 2507 images in val-all-04.txt
saved 10031 images in train-all-04.txt


In [8]:
import json
import os
import xml.etree.ElementTree as ET

def get_image_files_from_txt(txt_file_path):
    with open(txt_file_path, 'r') as file:
        lines = file.readlines()
    image_files = [line.strip().format('visible') for line in lines]
    return image_files

def xml_to_coco(xml_folder_path, image_files, category_mapping):
    coco_format = {
        "info": {
            "dataset": "KAIST Multispectral Pedestrian Benchmark",
            "url": "https://soonminhwang.github.io/rgbt-ped-detection/",
            "related_project_url": "http://multispectral.kaist.ac.kr",
            "publish": "CVPR 2015"
        },
        "info_improved": {
            "sanitized_annotation": {
                "publish": "BMVC 2018",
                "url": "https://li-chengyang.github.io/home/MSDS-RCNN/",
                "target": "files in train-all-02.txt (set00-set05)"
            },
            "improved_annotation": {
                "url": "https://github.com/denny1108/multispectral-pedestrian-py-faster-rcnn",
                "publish": "BMVC 2016",
                "target": "files in test-all-20.txt (set06-set11)"
            }
        },
        "images": [],
        "annotations": [],
        "categories": [{"id": id, "name": name} for id, name in category_mapping.items()]
    }
    
    image_id = 0
    annotation_id = 0
    
    for image_file in image_files:
        base_name = os.path.basename(image_file)
        xml_file = os.path.join(xml_folder_path, base_name.replace('.jpg', '.xml'))
        
        if not os.path.exists(xml_file):
            continue
        
        tree = ET.parse(xml_file)
        root = tree.getroot()
        
        filename = root.find('filename').text
        width = int(root.find('size/width').text)
        height = int(root.find('size/height').text)
        
        coco_format["images"].append({
            "id": image_id,
            "im_name": filename,
            "height": height,
            "width": width
        })
        
        for obj in root.findall('object'):
            category = obj.find('name').text
            category_id = category_mapping.get(category)
            
            if category_id is None:
                continue
            
            bbox = obj.find('bndbox')
            x = int(bbox.find('x').text)
            y = int(bbox.find('y').text)
            w = int(bbox.find('w').text)
            h = int(bbox.find('h').text)
            
            coco_format["annotations"].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": category_id,
                "bbox": [x, y, w, h],
                "height": h,
                "occlusion": 0,
                "ignore": 0
            })
            
            annotation_id += 1
        
        image_id += 1
    
    return coco_format

# Paths
xml_folder_path = 'datasets/kaist-rgbt/train/labels-xml'
val_txt_file_path = 'datasets/kaist-rgbt/val-all-04.txt'
output_val_json_path = 'utils/eval/KAIST_annotation.json'

# Category mapping
category_mapping = {
    "person": 0,
    "cyclist": 1,
    "people": 2,
    "person?": 3
}

val_image_files = get_image_files_from_txt(val_txt_file_path)

val_coco_json = xml_to_coco(xml_folder_path, val_image_files, category_mapping)

with open(output_val_json_path, 'w') as f:
    json.dump(val_coco_json, f, indent=4)
